In [1]:
# example making new folder with os
import os
import pandas as pd
import numpy as np
os.makedirs('Data/',exist_ok=True) # Confirm folder created
os.listdir("Data/")

['.ipynb_checkpoints',
 'title-akas-us-only (5).csv',
 'title.basics (1).tsv.gz',
 'title.ratings (1).tsv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

In [2]:
basics_url=("https://datasets.imdbws.com/title.basics.tsv.gz")
basics = pd.read_csv(basics_url,sep='\t', low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [3]:
ratings_url='https://datasets.imdbws.com/title.ratings.tsv.gz'
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1990
1,tt0000002,5.8,265
2,tt0000003,6.5,1856
3,tt0000004,5.5,178
4,tt0000005,6.2,2641


In [5]:
akas = pd.read_csv("Data/title-akas-us-only (5).csv", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
3,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
4,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


## reading akas file

###Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [6]:
   # Filter the basics table down to only include the US by using the filter akas dataframe
keepers = ratings['tconst'].isin(akas['titleId']) 
ratings = ratings[keepers]

In [8]:
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1990
1,tt0000002,5.8,265
4,tt0000005,6.2,2641
5,tt0000006,5.0,183
6,tt0000007,5.4,828
...,...,...,...
1336636,tt9916200,8.1,233
1336637,tt9916204,8.2,267
1336644,tt9916348,8.3,18
1336645,tt9916362,6.4,5464


- Include only movies that were released in the United States. 
- Exclude any movie with missing values for genre or runtime 
- Include only full-length movies (titleType = "movie").
- Include only movies that were released 2000 - 2021 (include 2000 and 2021) 
- Include only fictional movies (not from the Documentary genre)

## working on the basics data frame

####Replace "\N" with np.nan

In [10]:
basics.replace({'\\N':np.nan},inplace=True)
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1350798
endYear           9956055
runtimeMinutes    7075336
genres             452163
dtype: int64

####Eliminate movies that are null for runtimeMinutes

In [12]:
basics = basics.dropna(subset=['runtimeMinutes','genres','startYear'])
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear               0
endYear           2689747
runtimeMinutes          0
genres                  0
dtype: int64

include only full-length movies (titleType = "movie").

In [13]:
basics = basics.loc[basics['titleType']=='movie']

In [14]:
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
...,...,...,...,...,...,...,...,...,...
10066824,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,NaN,123,Drama
10066865,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,NaN,57,Documentary
10066892,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,NaN,100,Documentary
10066914,tt9916730,movie,6 Gunn,6 Gunn,0,2017,NaN,116,Drama


keep startYear 2000-2021

In [15]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
930,tt0000941,movie,Locura de amor,Locura de amor,0,1909,NaN,45,Drama


In [16]:
basics['startYear'] = basics['startYear'].astype(float)

C:\Users\awot1\AppData\Local\Temp\ipykernel_13112\3352754901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basics['startYear'] = basics['startYear'].astype(float)


In [17]:
basics=basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2021)]

In [18]:
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34801,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61113,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67487,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67665,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86792,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
10066656,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"
10066695,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020.0,NaN,84,Thriller
10066740,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History"
10066824,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019.0,NaN,123,Drama


Eliminate movies that include "Documentary" in genre

Keep only US movies

In [19]:
   # Filter the basics table down to only include the US by using the filter akas dataframe
keepers = basics['tconst'].isin(akas['titleId']) 
basics = basics[keepers]

In [20]:
basics 

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34801,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61113,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67487,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67665,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86792,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
10066112,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
10066507,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
10066647,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
10066656,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


Before saving, run a final .info() for each of the dataframes to show a summary of how many movies remain and the datatypes of each feature

In [21]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81896 entries, 34801 to 10066740
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81896 non-null  object 
 1   titleType       81896 non-null  object 
 2   primaryTitle    81896 non-null  object 
 3   originalTitle   81896 non-null  object 
 4   isAdult         81896 non-null  object 
 5   startYear       81896 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  81896 non-null  object 
 8   genres          81896 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.2+ MB


In [23]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452564 entries, 0 to 1452563
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1452564 non-null  object
 1   ordering         1452564 non-null  int64 
 2   title            1452564 non-null  object
 3   region           1452564 non-null  object
 4   language         1452564 non-null  object
 5   types            1452564 non-null  object
 6   attributes       1452564 non-null  object
 7   isOriginalTitle  1452564 non-null  object
dtypes: int64(1), object(7)
memory usage: 88.7+ MB


In [24]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504602 entries, 0 to 1336650
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         504602 non-null  object 
 1   averageRating  504602 non-null  float64
 2   numVotes       504602 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.4+ MB


Save each file to a compressed csv file "Data/" folder inside your repository.

In [25]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [26]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [31]:
## Save current dataframe to file.
akas.to_csv("Data/title-akas-us-only (5).csv.gz",compression='gzip',index=False)

In [32]:
# Open saved file and preview again
akas = pd.read_csv("Data/title-akas-us-only (5).csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
3,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
4,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


In [36]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',mode='a',index=False)

In [35]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1990
1,tt0000002,5.8,265
2,tt0000003,6.5,1856
3,tt0000004,5.5,178
4,tt0000005,6.2,2641
